#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2022, 3, 11)

### Yesterday = Last business day

In [3]:
#yesterday = date(2022, 2, 11)
yesterday

datetime.date(2022, 3, 11)

### Restart and Run All Cells

In [4]:
cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

In [5]:
format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [6]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1114838,WHART,2022-03-04,2022-03-11,5,12.40,11.80,-0.60,-4.84%,-6,-2,12.30,11.70,"1,121,300",2022-03-12,2022-03-12,1


In [7]:
names = tmp["name"]
name = names.to_string(index=False)
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,11.40,I,0.00,12.40,17.84,0.22,12.30,"20,000",-5,4,"20,000",0,0,DOS,SET


### End of Tables in the process

In [8]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '2022-03-11' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [9]:
df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['diff'] = (df['to_price'] - df['fm_price'])
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
name,,,,,,,,,,,
KCE,2022-03-08,2022-03-11,54.75,58.50,"60,888,462",60.50,54.75,B,SET50,3.75,6.85%
MAKRO,2022-03-09,2022-03-11,41.75,41.50,"11,373,946",41.75,41.25,B,SET,-0.25,-0.60%
MCS,2022-03-08,2022-03-11,14.40,14.70,"7,994,808",14.80,14.30,B,SET,0.30,2.08%
RCL,2022-03-08,2022-03-11,48.00,50.00,"27,011,792",50.50,47.00,B,SET100,2.00,4.17%
STA,2022-03-08,2022-03-11,24.90,26.25,"40,905,412",26.25,24.20,B,SET100,1.35,5.42%


### Price change up or down more than 3%

In [10]:
#df['bdays'] = np.busday_count(df['fm_date'],df['to_date'])
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.percent) >= 3,
    ['percent','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
KCE,6.85%,B,54.75,58.50,SET50,2022-03-08,3,60.50,54.75
STA,5.42%,B,24.90,26.25,SET100,2022-03-08,3,26.25,24.20
RCL,4.17%,B,48.00,50.00,SET100,2022-03-08,3,50.50,47.00
HREIT,3.53%,I,8.50,8.80,SET,2022-03-09,2,8.80,8.55
PTT,3.27%,I,38.25,39.50,SET50,2022-03-08,3,39.75,38.00
IP,3.26%,I,18.40,19.00,mai,2022-03-08,3,19.40,18.50
WHART,-4.84%,I,12.40,11.80,SET,2022-03-04,7,12.30,11.70
ASP,-7.89%,I,3.80,3.50,SET,2022-03-09,2,3.62,3.44


In [11]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

watch_list.sort_values(['percent'],ascending=[False]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(box_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(one_file)

### Create daily-sales from sales

In [12]:
df[cols].head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
name,,,,,,,,,
KCE,2022-03-08,2022-03-11,54.75,58.50,"60,888,462",60.50,54.75,6.85%,B
MAKRO,2022-03-09,2022-03-11,41.75,41.50,"11,373,946",41.75,41.25,-0.60%,B
MCS,2022-03-08,2022-03-11,14.40,14.70,"7,994,808",14.80,14.30,2.08%,B
RCL,2022-03-08,2022-03-11,48.00,50.00,"27,011,792",50.50,47.00,4.17%,B
STA,2022-03-08,2022-03-11,24.90,26.25,"40,905,412",26.25,24.20,5.42%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(box_file)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file)

### Create monitors from stocks

In [14]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.head()

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DCC,B


In [16]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
WHERE trade IN ("B","I","O","S")
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,9
1,S,13


### output_file = iCloud Drive on PC

In [17]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [18]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,9
1,S,13


In [19]:
sql = """
SELECT name, trade, market
FROM orders 
ORDER BY trade, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,trade,market
name,,
DCC,B,SET
KCE,B,SET50
MAKRO,B,SET
MCS,B,SET
RCL,B,SET100
STA,B,SET100
SYNEX,B,SET100
TISCO,B,SET50
TMT,B,SET


In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [21]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
DCC,B
KCE,B
MAKRO,B
MCS,B
RCL,B
STA,B
SYNEX,B
TISCO,B
TMT,B


In [22]:
buy_candidates.shape[0]

9

In [23]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("I","S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DOHOME,S
HREIT,S
IP,S
JASIF,S
PTT,S


In [24]:
sell_candidates.shape[0]

13